# <span><center>Microproyecto 2: Identificación de relaciones semánticas de los Objetivos de Desarrollo Sostenible (ODSG)</center></span>

## Equipo 16: Mateo Hernandez Gualdron - Martin Alejandro Quintero Rincon

#### Universidad de los Andes, Maestría en Inteligencia Artificial.
#### Machine Learning No Supervisado (MLNS)
______________________________________________________________________________________________________________________________________________

